In [1]:
import pymp
from multiprocessing import Manager, cpu_count 
import sys
import os
import glob 
import gezi
from gezi import tqdm
import pandas as pd
import json
import tensorflow as tf
from absl import flags
FLAGS = flags.FLAGS
import melt

from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.offline as py
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

#setting offilne
plotly.offline.init_notebook_mode(connected=True)
tf.enable_eager_execution()

from projects.feed.rank.src import config
from projects.feed.rank.src.model import Model
from projects.feed.rank.src.tfrecord_dataset import Dataset

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:10000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
product='sgsapp'
mark='video'

In [4]:
# 载入模型
model_hour = '2020051810'
valid_hour = '2020051812'
# valid_hour = '2020030812'
model_root = f'/search/odin/publicData/CloudS/mkyuwen/rank_shareKwEmb/data/video_hour_sgsapp_v1/models/15'
# model_root = '/home/gezi/tmp/rank/exps/tuwen/v29/dlrm-norm/'
model_dir = f'{model_root}/{model_hour}'
# model_dir = model_root
tfrecord_root = f'/search/odin/publicData/CloudS/yuwenmengke/rank_0430/sgsapp/data/video_hour_sgsapp_v1/tfrecords'
tfrecord_dir = f'{tfrecord_root}/{valid_hour}'
print('model:', model_dir)
print('record:', tfrecord_dir)
argv = open(f'{model_dir}/command.txt').readline().strip().replace('data_version=2', 'data_version=1').split()
FLAGS(argv)
FLAGS.eager = True
# FLAGS.fields_norm=True
config.init()
model = Model()
melt.eager.restore(model, model_dir)

model: /search/odin/publicData/CloudS/mkyuwen/rank_shareKwEmb/data/video_hour_sgsapp_v1/models/15/2020051810
record: /search/odin/publicData/CloudS/yuwenmengke/rank_0430/sgsapp/data/video_hour_sgsapp_v1/tfrecords/2020051812


2020-05-18 14:46:43 0:00:00 Tf dataset and Tf model train in Eager mode, keras False, distributed:False
2020-05-18 14:46:43 0:00:00 log_level: 20 (try --debug to show more or --log_level=(> 20) to show less(no INFO), try --verbose to show train/valid loss intervaly)
2020-05-18 14:46:43 0:00:00 batch_size: 512 eval_batch_size: 512 batch_size_per_gpu: 512 num_gpus: 0 gpu: [] CUDA_VISIABLE_DEVICES=[-1] work_mode: train distributed: False horovod: False
2020-05-18 14:46:43 0:00:00 model: [15] model_dir: [/search/odin/mkyuwen/rank_shareKwEmb/video/model/15]
2020-05-18 14:46:44 0:00:00 Latest checkpoint: /search/odin/publicData/CloudS/mkyuwen/rank_shareKwEmb/data/video_hour_sgsapp_v1/models/15/2020051810/model.ckpt-2020051810-1.00-2582


In [5]:
dataset = Dataset('valid')

In [6]:
files = gezi.list_files(tfrecord_dir)
total = melt.get_num_records(files) 
total

536530

In [7]:
batch_size = 1
batches = dataset.make_batch(batch_size=batch_size, filenames=files, repeat=False)
num_steps = -int(-total // batch_size)

In [21]:
for i, (x, _) in enumerate(batches):
  if i == 10:
    break

In [22]:
x

{'doc_keyword': <tf.Tensor: id=3300, shape=(1, 0), dtype=int64, numpy=array([], shape=(1, 0), dtype=int64)>,
 'doc_kw_feed': <tf.Tensor: id=3301, shape=(1, 4), dtype=int64, numpy=
 array([[-2071810834465357832, -9043944592419789055,  6875087379715033459,
         -8482651893393409696]])>,
 'doc_topic': <tf.Tensor: id=3302, shape=(1, 1), dtype=int64, numpy=array([[0]])>,
 'field': <tf.Tensor: id=3305, shape=(1, 648), dtype=int64, numpy=
 array([[ 5260721114193651035, -8919556150788677908, -2869353143095762574,
         -8125370836086984651, -8125370836086984651, -8125370836086984651,
         -8125370836086984651, -8125370836086984651, -8125370836086984651,
         -8125370836086984651, -8125370836086984651, -6160126929489305482,
         -6160126929489305482, -6160126929489305482, -6160126929489305482,
         -6160126929489305482, -6160126929489305482, -6160126929489305482,
         -6160126929489305482, -6160126929489305482, -6160126929489305482,
         -6160126929489305482, -616

In [23]:
x['model_ver']

<tf.Tensor: id=3319, shape=(1,), dtype=string, numpy=array([b'2020051809'], dtype=object)>

In [24]:
x['ol_pred']

<tf.Tensor: id=3323, shape=(1,), dtype=float32, numpy=array([0.0207], dtype=float32)>

In [25]:
model(x)

<tf.Tensor: id=4016, shape=(1, 1), dtype=float32, numpy=array([[0.0178]], dtype=float32)>

In [26]:
model(x)

<tf.Tensor: id=4596, shape=(1, 1), dtype=float32, numpy=array([[0.0178]], dtype=float32)>

In [27]:
x['mid']

<tf.Tensor: id=3313, shape=(1,), dtype=string, numpy=array([b'5789867368034317068'], dtype=object)>

In [28]:
x['docid']

<tf.Tensor: id=3303, shape=(1,), dtype=string, numpy=array([b'open_doc_prod29111828'], dtype=object)>

In [29]:
x['model_ver']

<tf.Tensor: id=3319, shape=(1,), dtype=string, numpy=array([b'2020051809'], dtype=object)>

In [30]:
model.prob_click

<tf.Tensor: id=4533, shape=(1, 1), dtype=float32, numpy=array([[0.0866]], dtype=float32)>

In [31]:
model.prob_dur

<tf.Tensor: id=4593, shape=(1, 1), dtype=float32, numpy=array([[0.2821]], dtype=float32)>